In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))                               
df = df.drop('_id', 1)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' #Grazioso Salvare’s logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Center(
        html.A(href='https://www.snhu.edu',children=[
            html.Img(alt="Grazioso Salvare logo",
                    src='data:image/png;base64,{}'.format(encoded_image.decode()), # Grazioso Salvare Logo
                )
            ]
        )
    ),
    html.Center(html.B(html.H1('Maya Neely AAC Dashboard'))),
    html.Hr(),
    
    # Radio Button List
    html.Div([
        dcc.RadioItems(
        id = 'filters',
        options = [           
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster Rescue ', 'value': 'DISASTER'},  
            {'label': 'Reset', 'value': 'RESET'},
        ],
        value = 'RESET',
        labelStyle = {'display': 'inline-block'}
    ),
    ]),
    html.Br(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,        
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', 'value')]
)
def animalSort(rescueAnimal):
    if rescueAnimal == "WATER":
            df = pd.DataFrame.from_records(shelter.read({'$and':
                                                             [{"sex_upon_outcome":"Intact Female"},
                                                             {'breed':{'$in':["Labrador Retriever Mix",
                                                                              "Chesapeake Bay Retriever",
                                                                              "Newfoundland"]}},
                                                             {"age_upon_outcome_in_weeks":{'$gte':26, '$lte':156}}
                                                             ]}))
            df = df.drop('_id', 1)
    elif rescueAnimal == "MOUNTAIN":
            df = pd.DataFrame.from_records(shelter.read({'$and': 
                                                        [{"breed":{'$in':["German Shepherd",
                                                                          "Alaskan Malamute",
                                                                          "Old English Sheepdog",
                                                                          "Siberian Husky",
                                                                          "Rottweiler"]}}, 
                                                         {"sex_upon_outcome":"Intact Male"}, 
                                                         {"age_upon_outcome_in_weeks":{'$gte':26, '$lte':156}}
                                                        ]}))
            df = df.drop('_id', 1)
    elif rescueAnimal == "DISASTER":
            df = pd.DataFrame.from_records(shelter.read({'$and':
                                                        [{"breed":{'$in':["Doberman Pinscher",
                                                                          "German Shepherd",
                                                                          "Golden Retriever",
                                                                          "Bloodhound",
                                                                          "Rottweiler"]}}, 
                                                         {"sex_upon_outcome":"Intact Male"}, 
                                                         {"age_upon_outcome_in_weeks":{'$gte':20, '$lte':300}}
                                                        ]}))
            df = df.drop('_id', 1)
    elif rescueAnimal == "RESET":
            df = pd.DataFrame.from_records(shelter.read({}))
            df = df.drop('_id', 1)
    else:
            df = pd.DataFrame.from_records(shelter.read({})) #default value is to show unfiltered table
            df = df.drop('_id', 1)
        
    return df.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #Create values for breeds and count
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #Create pie chart
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names,                 
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app